### Overgear Scrapper

In [1]:
game_link = "https://overgear.com/games/battlefield-6"


In [3]:
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
import time

# Configure Selenium Edge options
options = EdgeOptions()
options.page_load_strategy = "eager"
# options.add_argument("--headless=new")  # uncomment to run without browser window
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")

# Start driver
driver = webdriver.Edge(options=options)

try:
    url = game_link
    driver.get(url)

    # wait a bit for content to load
    time.sleep(3)

    # get page source
    html = driver.page_source

    # parse with BeautifulSoup
    soup = BeautifulSoup(html, "html.parser")

    # print the soup variable
    print(soup.prettify())  # just print first 2000 chars so console isn’t overwhelmed
finally:
    driver.quit()


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


<html class="bg-neutral-color2" lang="en" style="--device-vh: 715px;">
 <head>
  <meta content="#53528b" name="theme-color"/>
  <link crossorigin="use-credentials" href="/manifest.json" rel="manifest"/>
  <link href="/favicon.ico" rel="icon" sizes="32x32"/>
  <link href="/favicon.svg" rel="icon" type="image/svg+xml"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
  <meta charset="utf-8"/>
  <meta content="!" name="fragment"/>
  <meta content="Buy Battlefield 6 Boosting and get the desired amount of currency, raid boosts, leveling, and item boosts. Hire a pro to make your DF experience even better." data-rh="true" name="description"/>
  <meta content="/assets-image/a12d317083e7b8affea0f79fe4a753f7-og.png" data-rh="true" itemprop="image"/>
  <meta content="367370773695386" data-rh="true" property="fb:admins"/>
  <meta content="158774354866279" data-rh="true" property="fb:app_id"/>
  <meta content="/assets-image/a12d317083e7b8affea0f79fe4a753f7-og.png" data-rh="true" name=

In [4]:
# --- Extract first service link from soup ---
service_links = []

# Find all "category__item" anchors (each service link)
for a in soup.find_all("a", class_="category__item"):
    href = a.get("href")
    if href and href.startswith("/"):
        full_url = "https://overgear.com" + href
        service_links.append(full_url)

# Debug: print all links found
print("Found service links:", service_links)

# Take only the first one for now (break/limit)
if service_links:
    test_service_url = service_links[0]
    print("\nOpening single service link:", test_service_url)


Found service links: ['https://overgear.com/games/battlefield-6/rent-a-booster', 'https://overgear.com/games/battlefield-6/level-boost', 'https://overgear.com/games/battlefield-6/weapon-leveling']

Opening single service link: https://overgear.com/games/battlefield-6/rent-a-booster


In [7]:
# --- Open the first service link and build BeautifulSoup for that page ---

from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.edge.options import Options as EdgeOptions
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
import time

def get_soup_via_selenium(url: str, headless: bool = False, wait_css: str = "h1"):
    """Open a URL in Edge via Selenium, wait for a key element, and return BeautifulSoup(html)."""
    opts = EdgeOptions()
    opts.page_load_strategy = "eager"
    # opts.add_argument("--headless=new")  # uncomment to run headless
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")

    driver = webdriver.Edge(options=opts)
    try:
        driver.get(url)
        # Wait for something meaningful to appear (fallback to sleep if it doesn't)
        try:
            WebDriverWait(driver, 12).until(EC.presence_of_element_located((By.CSS_SELECTOR, wait_css)))
        except Exception:
            time.sleep(3)

        html = driver.page_source
        return BeautifulSoup(html, "html.parser")
    finally:
        driver.quit()

# Guard: ensure previous cell set `test_service_url`
if 'test_service_url' not in globals() or not test_service_url:
    raise RuntimeError("No test_service_url set. Run the previous cell to collect service links.")

print("Opening:", test_service_url)
service_soup = get_soup_via_selenium(test_service_url)

# Preview the page HTML (first 2000 chars so the console isn't flooded)
print(service_soup.prettify())

# Optional: quick sanity checks (title/price if present)
title_tag = service_soup.find("h1")
price_tag = (
    service_soup.select_one("[data-qa='product-price']") or
    service_soup.select_one(".product-price, .price__current, .product-price__current")
)

print("\nParsed fields:")
print("Title:", title_tag.get_text(strip=True) if title_tag else "N/A")
print("Price:", price_tag.get_text(strip=True) if price_tag else "N/A")


There was an error managing msedgedriver (error sending request for url (https://msedgedriver.azureedge.net/LATEST_RELEASE_139_LINUX)); using driver found in the cache


Opening: https://overgear.com/games/battlefield-6/rent-a-booster
<html class="bg-neutral-color2" lang="en" style="--device-vh: 715px;">
 <head>
  <meta content="#53528b" name="theme-color"/>
  <link crossorigin="use-credentials" href="/manifest.json" rel="manifest"/>
  <link href="/favicon.ico" rel="icon" sizes="32x32"/>
  <link href="/favicon.svg" rel="icon" type="image/svg+xml"/>
  <link href="/apple-touch-icon.png" rel="apple-touch-icon"/>
  <meta charset="utf-8"/>
  <meta content="!" name="fragment"/>
  <meta content="BF6 Rental Service — Need a hand? Our boosters can handle various tasks! Leveling, weapons unlock, beta rewards. PC, Xbox, PS. 24/7 Online Support" data-rh="true" name="description"/>
  <meta content="/assets-image/a12d317083e7b8affea0f79fe4a753f7-og.png" data-rh="true" itemprop="image"/>
  <meta content="367370773695386" data-rh="true" property="fb:admins"/>
  <meta content="158774354866279" data-rh="true" property="fb:app_id"/>
  <meta content="/assets-image/a12d317

In [11]:
# --- Parse product details & options from `service_soup` ---

import json
import re
from bs4 import BeautifulSoup

def clean_text(s: str) -> str:
    return re.sub(r"\s+", " ", (s or "").strip())

def parse_money(txt: str):
    """Return (amount:str like '15.00', currency:str or None) from a money-like text."""
    if not txt:
        return ("", None)
    t = txt.strip()
    currency = None
    if t.startswith("$"):
        currency = "USD"
    # Keep only digits, comma, dot, minus
    num = re.sub(r"[^\d,.\-]", "", t)
    if "," in num and "." in num:
        num = num.replace(",", "")
    elif "," in num and "." not in num:
        num = num.replace(",", ".")
    try:
        val = f"{float(num):.2f}"
        return (val, currency)
    except Exception:
        return ("", currency)

def get_jsonld_product(soup: BeautifulSoup):
    """Extract the first Product JSON-LD dict if available."""
    for script in soup.find_all("script", type="application/ld+json"):
        raw = script.string or ""
        if not raw.strip():
            continue
        try:
            data = json.loads(raw)
        except Exception:
            continue
        candidates = data if isinstance(data, list) else [data]
        for d in candidates:
            if isinstance(d, dict) and d.get("@type") in ("Product", ["Product"]):
                return d
    return None

def nearest_section_title(el: BeautifulSoup):
    """Find the nearest preceding section label span used as a title (e.g., 'Boost method')."""
    lab = el.find_previous("span", class_="font-main text-body-s font-medium text-neutral-color6")
    return clean_text(lab.get_text()) if lab else None

# 1) Title
title = None
h1 = service_soup.find("h1")
if h1:
    title = clean_text(h1.get_text())

# 2) JSON-LD
product_ld = get_jsonld_product(service_soup) or {}
ld_name = product_ld.get("name")
ld_desc = product_ld.get("description")
ld_image = product_ld.get("image")
ld_offers = product_ld.get("offers") or {}
ld_price = ld_offers.get("Price") or ld_offers.get("price")
ld_currency = ld_offers.get("priceCurrency")

# 3) Price (fallback to widget display if JSON-LD absent)
price_text_node = service_soup.select_one(
    ".PurchaseWidgetStickyWrap__StyledFlex-sc-2d5n5g-0 .heading-base"
)
price_val, price_currency = parse_money(price_text_node.get_text(strip=True)) if price_text_node else ("","")

# Prefer JSON-LD when present
if ld_price:
    try:
        price_val = f"{float(ld_price):.2f}"
    except Exception:
        pass
if ld_currency:
    price_currency = ld_currency or price_currency

# 4) Image (prefer JSON-LD, else right-side product image)
image_url = ld_image
if not image_url:
    img = service_soup.select_one("form.product-form__StyledForm-sc-ig4zxp-0 img[alt]")
    if img and img.has_attr("src"):
        image_url = img["src"]

# 5) Description (prefer JSON-LD, else the first description block)
description = ld_desc
if not description:
    desc_block = service_soup.select_one(".style-content-v3__StyleContentV3-sc-1uqk2fg-0")
    if desc_block:
        description = clean_text(desc_block.get_text(" ", strip=True))

# 6) Options parsing
options = []

# 6a) Hours slider
hours_default = None
hours_min = None
hours_max = None
hours_input = service_soup.select_one("input#currency")
if hours_input:
    hours_default = hours_input.get("value", "").strip() or None

slider_handle = service_soup.select_one(".rc-slider .rc-slider-handle")
if slider_handle:
    try:
        vmin = slider_handle.get("aria-valuemin")
        vmax = slider_handle.get("aria-valuemax")
        if vmin is not None:
            hours_min = int(vmin)
        if vmax is not None:
            hours_max = int(vmax)
    except Exception:
        pass

if hours_default or (hours_min is not None) or (hours_max is not None):
    options.append({
        "group": "Number of hours",
        "type": "slider",
        "name": "number_of_hours",
        "default": hours_default,
        "min": hours_min,
        "max": hours_max,
        "choices": None,  # sliders do not use choices
    })

# 6b) Radios/checkboxes grouped by nearest section title
for lab in service_soup.select("form label"):
    inp = lab.find("input")
    if not inp:
        continue
    itype = (inp.get("type") or "").lower()
    if itype not in ("radio", "checkbox"):
        continue

    # Visible text on the clickable box
    txt = clean_text(lab.get_text(" ", strip=True))
    # Extract the first price/percent modifier if present
    m = re.search(r"(\+\s*\$?\d+(?:[.,]\d+)?%?)", txt)
    modifier_text = m.group(1) if m else None
    # Remove modifier from label
    label_text = clean_text(txt.replace(modifier_text or "", "")) or ""

    group_title = nearest_section_title(lab) or "Options"

    # Find a matching non-slider group with the same title
    group = next(
        (g for g in options if g["group"] == group_title and g.get("type") != "slider"),
        None
    )

    # If not found, create it
    if not group:
        group = {
            "group": group_title,
            "type": itype,  # radio or checkbox
            "name": re.sub(r"[^a-z0-9_]+", "_", group_title.lower()).strip("_") or "options",
            "default": None,
            "min": None,
            "max": None,
            "choices": []
        }
        options.append(group)
    else:
        # Ensure choices list exists even if something overwrote it
        if group.get("choices") is None:
            group["choices"] = []
        # If any label in this group is a radio, unify the group type to radio
        if itype == "radio":
            group["type"] = "radio"

    # Default detection: if input has checked attribute
    is_default = inp.has_attr("checked")
    if is_default and group["type"] == "radio":
        group["default"] = label_text

    group["choices"].append({
        "value": inp.get("value", "") or "",
        "label": label_text,
        "price_modifier": modifier_text
    })

# Build final record
record = {
    "url": test_service_url if 'test_service_url' in globals() else None,
    "title": title or ld_name,
    "price": price_val or (f"{float(ld_price):.2f}" if ld_price else ""),
    "currency": price_currency or "USD",
    "image": image_url,
    "description": description,
    "options": options
}

# Pretty print a concise summary
print("TITLE:", record["title"])
print("PRICE:", record["currency"], record["price"])
print("IMAGE:", record["image"])
print("DESC (first 200 chars):", (record["description"] or "")[:200], "...")
print("\nOPTION GROUPS:")
for g in record["options"]:
    if g["type"] == "slider":
        print(f" - {g['group']} [slider] default={g['default']} min={g['min']} max={g['max']}")
    else:
        ch = g.get("choices") or []
        print(f" - {g['group']} [{g['type']}] choices={len(ch)} default={g.get('default')}")
        for c in ch[:3]:
            print("    •", c["label"], "modifier:", c["price_modifier"])

# Keep the full structure for later steps
service_record = record


TITLE: Battlefield 6 Rent a Booster
PRICE: USD 15.00
IMAGE: https://overgear.com/cdn/uploads/f9ae50a40c339550e80f132caeb33241.jpeg
DESC (first 200 chars): BF6 Rental Service — Need a hand? Our boosters can handle various tasks! Leveling, weapons unlock, beta rewards. PC, Xbox, PS. 24/7 Online Support ...

OPTION GROUPS:
 - Number of hours [slider] default=1 min=0 max=9
 - Number of hours [radio] choices=7 default=None
    • Piloted modifier: None
    • Self-play modifier: +50%
    • One modifier: None


In [15]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Notebook-ready Overgear scraper with Selenium.
Usage:
    game_link = "https://www.overgear.com/games/..."
    rec = scrape_product(game_link)
    rec   # dict with title, price, options etc.
"""

import json, re, time
from typing import Any, Dict, List, Optional, Tuple
from urllib.parse import urljoin, urlparse
from bs4 import BeautifulSoup

# ------------------ Soup helpers ------------------

PARSER = "html.parser"
def get_soup(html: str) -> BeautifulSoup:
    return BeautifulSoup(html, PARSER)

def clean_text(s: str) -> str:
    import re
    return re.sub(r"\s+", " ", (s or "").strip())

def parse_money(txt: str) -> Tuple[str, Optional[str]]:
    if not txt:
        return ("", None)
    t = txt.strip()
    currency = None
    if t.startswith("$"):
        currency = "USD"
    num = re.sub(r"[^\d,.\-]", "", t)
    if "," in num and "." in num:
        num = num.replace(",", "")
    elif "," in num and "." not in num:
        num = num.replace(",", ".")
    try:
        val = f"{float(num):.2f}"
        return (val, currency)
    except Exception:
        return ("", currency)

# ------------------ Selenium loader ------------------

from selenium import webdriver
from selenium.webdriver.chrome.options import Options as ChromeOptions
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException

UA = (
    "Mozilla/5.0 (Windows NT 10.0; Win64; x64) "
    "AppleWebKit/537.36 (KHTML, like Gecko) "
    "Chrome/124.0.0.0 Safari/537.36"
)

def build_chrome(headless: bool = True, driver_path: Optional[str] = None) -> webdriver.Chrome:
    opts = ChromeOptions()
    if headless:
        opts.add_argument("--headless=new")
    opts.add_argument("--disable-gpu")
    opts.add_argument("--no-sandbox")
    opts.add_argument("--disable-dev-shm-usage")
    opts.add_argument("--window-size=1400,900")
    opts.add_argument(f"--user-agent={UA}")
    opts.add_argument("--lang=en-US")
    opts.add_experimental_option("excludeSwitches", ["enable-automation"])
    opts.add_experimental_option("useAutomationExtension", False)

    if driver_path:
        driver = webdriver.Chrome(driver_path, options=opts)
    else:
        driver = webdriver.Chrome(options=opts)
    return driver

def fetch_with_selenium(driver, url: str, wait_seconds: int = 25) -> str:
    driver.get(url)
    # small scroll
    try:
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(1)
        driver.execute_script("window.scrollTo(0, 0);")
    except Exception:
        pass
    try:
        WebDriverWait(driver, wait_seconds).until(
            lambda d: d.find_element(By.TAG_NAME, "h1")
        )
    except TimeoutException:
        pass
    return driver.page_source

# ------------------ JSON-LD + product parse ------------------

def get_jsonld_product(soup: BeautifulSoup) -> Optional[Dict[str, Any]]:
    for script in soup.find_all("script", type="application/ld+json"):
        raw = script.string or ""
        if not raw.strip():
            continue
        try:
            data = json.loads(raw)
        except Exception:
            continue
        candidates = data if isinstance(data, list) else [data]
        for d in candidates:
            if isinstance(d, dict) and d.get("@type") == "Product":
                return d
    return None

def parse_product_page(html: str, base_url: str) -> Dict[str, Any]:
    soup = get_soup(html)

    # Title
    h1 = soup.find("h1")
    title = clean_text(h1.get_text()) if h1 else None

    # JSON-LD
    product_ld = get_jsonld_product(soup) or {}
    ld_name = product_ld.get("name")
    ld_desc = product_ld.get("description")
    ld_image = product_ld.get("image")
    ld_offers = product_ld.get("offers") or {}
    ld_price = ld_offers.get("price") or ld_offers.get("Price")
    ld_currency = ld_offers.get("priceCurrency")

    # Price fallback
    price_val, price_currency = ("", None)
    price_node = soup.select_one(".heading-base")
    if price_node:
        price_val, price_currency = parse_money(price_node.get_text(strip=True))
    if ld_price:
        try:
            price_val = f"{float(ld_price):.2f}"
        except: pass
    if ld_currency:
        price_currency = ld_currency

    # Image
    image_url = ld_image
    if not image_url:
        img = soup.find("img", alt=True)
        if img and img.has_attr("src"):
            image_url = urljoin(base_url, img["src"])

    # Description
    description = ld_desc or None
    if not description:
        desc_block = soup.select_one(".style-content-v3__StyleContentV3-sc-1uqk2fg-0")
        if desc_block:
            description = clean_text(desc_block.get_text(" ", strip=True))

    return {
        "url": base_url,
        "title": title or ld_name,
        "price": price_val,
        "currency": price_currency or "USD",
        "image": image_url,
        "description": description,
    }

# ------------------ Top-level scrape fn ------------------

def scrape_product(game_link: str, headless: bool = True) -> Dict[str, Any]:
    driver = build_chrome(headless=headless)
    try:
        html = fetch_with_selenium(driver, game_link)
        rec = parse_product_page(html, base_url=game_link)
        return rec
    finally:
        driver.quit()


In [16]:
# 1. Put above cell in Jupyter and run it.

# 2. Then set your link:
# game_link = "https://www.overgear.com/games/destiny-2/..."

# 3. Call the scraper:
record = scrape_product(game_link, headless=False)
record


{'url': 'https://overgear.com/games/battlefield-6',
 'title': 'Battlefield 6 Boost',
 'price': '6.00',
 'currency': 'USD',
 'image': 'https://overgear.com/cdn-cgi/image/width=1919,quality=85,format=auto/cdn/files/82f06f19-51bf-4acd-b3ae-e8bd16327452',
 'description': 'Skip the grind by buying cheap BF6 boosts! Get all in-game unlocks for weapons and vehicles, level boosts, and more — all at unbeatable prices and very fast. We offer every type of boost you need to enjoy your games with the desired gear fast and play smarter. Use Battlefield 6 boosting services to ease your life.'}